In [44]:
import os
import subprocess
import pandas as pd
import platform
import pathlib
import numpy as np
from annotations import *
from extract_video_features import *
from extract_audio_features import *
import cv2
import seaborn as sns
from scipy.stats import kurtosis, skew
from sklearn.metrics import f1_score
import importlib
from tqdm import tqdm
import ordinal_classification as o_c
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from util import runGridSearchClassifiers
from sklearn.model_selection import cross_val_predict

In [2]:
#get current directory
cwd = os.getcwd()

In [3]:
cwd

'/Users/valentinadiproietto/filrouge'

In [4]:
OpenFace_folder = '/Users/valentinadiproietto/OpenFace'
filename_annotations = 'https://docs.google.com/spreadsheets/d/1Rqu1sJiD-ogc4a6R491JTiaYacptOTqh6DKqhwTa8NA/gviz/tq?tqx=out:csv&sheet=Template'

In [5]:
Video_folder = '/Users/valentinadiproietto/Desktop/video_stress'

In [6]:
video_paths, video_names = get_videos(Video_folder)

In [7]:
video_names

['WIN_20210331_21_22_52_Pro',
 'WIN_20210329_14_13_45_Pro',
 'WIN_20210406_18_49_10_Pro',
 'WIN_20210408_14_11_32_Pro',
 'WIN_20210408_15_20_51_Pro',
 'WIN_20210404_10_58_27_Pro',
 'WIN_20210414_06_24_52_Pro',
 'WIN_20210406_15_06_15_Pro',
 'WIN_20210417_14_53_12_Pro',
 'WIN_20210413_15_38_01_Pro',
 'WIN_20210408_11_48_58_Pro',
 'WIN_20210408_16_04_32_Pro',
 'WIN_20210329_10_16_02_Pro',
 'WIN_20210323_19_17_40_Pro',
 'WIN_20210409_10_26_11_Pro',
 'Test_pour_AFPA',
 'WIN_20210405_15_09_16_Pro',
 'WIN_20210407_14_54_56_Pro_edit2',
 'WIN_20210406_21_05_52_Pro',
 'WIN_20210403_18_49_15_Pro',
 'WIN_20210408_14_02_19_Pro',
 'WIN_20210415_15_41_24_Pro',
 'WIN_20210406_18_35_52_Pro',
 'WIN_20210402_14_27_50_Pro',
 'WIN_20210407_09_04_05_Pro',
 'WIN_20210402_19_04_53_Pro',
 'WIN_20210416_08_06_54_Pro',
 'Video_1',
 'WIN_20210408_14_00_44_Pro',
 'WIN_20210404_21_41_12_Pro',
 'WIN_20210330_13_10_29_Pro']

In [8]:
list_dataframes = []
for i in video_names: 
    list_dataframes.append(create_dataframe_video('/Users/valentinadiproietto/OpenFace/processed/', i))


In [9]:
video_names.remove('WIN_20210329_14_13_45_Pro')
video_names.remove('WIN_20210402_14_27_50_Pro')


In [10]:
len(video_names)

29

In [11]:
list_df_max = []
for v_name in tqdm(video_names):
    df_annoted = get_df_video_with_annotations('/Users/valentinadiproietto/OpenFace/processed/', v_name, filename_annotations, "max")
    list_df_max.append(eliminate_features(df_annoted))


    

100%|██████████| 29/29 [02:39<00:00,  5.50s/it]


## RECUPERO LE PREDIZIONI SULLE TIME WINDOW

In [56]:
predict5s.to_csv('all_features_tw5.csv')

In [57]:
predict5s = pd.read_csv ("all_features_tw5.csv')

,video_name,diapo,frameTimeWindow,ypredict
0,Test_pour_AFPA,9,44,0.0
1,Test_pour_AFPA,9,45,0.0
2,Test_pour_AFPA,9,46,0.0
3,Test_pour_AFPA,9,47,0.0
4,Test_pour_AFPA,9,48,0.0
...,...,...,...,...
5136,WIN_20210417_14_53_12_Pro,18,120,1.0
5137,WIN_20210417_14_53_12_Pro,18,121,1.0
5138,WIN_20210417_14_53_12_Pro,18,122,1.0
5139,WIN_20210417_14_53_12_Pro,18,123,1.0


In [15]:
predict5s = pd.read_csv('all_features_tw5.csv').drop("Unnamed: 0", axis =1)
predict5s

,video_name,diapo,frameTimeWindow,ypredict
0,Test_pour_AFPA,9,44,0.0
1,Test_pour_AFPA,9,45,0.0
2,Test_pour_AFPA,9,46,0.0
3,Test_pour_AFPA,9,47,0.0
4,Test_pour_AFPA,9,48,0.0
...,...,...,...,...
5136,WIN_20210417_14_53_12_Pro,18,120,1.0
5137,WIN_20210417_14_53_12_Pro,18,121,1.0
5138,WIN_20210417_14_53_12_Pro,18,122,1.0
5139,WIN_20210417_14_53_12_Pro,18,123,1.0


## PIVOT TABLE WITH COUNT OF PREDICTION 5SEC, TO PREDICT STRESS Global

In [18]:
X = predict5s.pivot_table(values='frameTimeWindow', columns='ypredict', index=['video_name'], aggfunc='count', fill_value=0)
X_sum = X.sum(axis=1).values.copy()
for col_number in range(len(X.columns)):
    X.iloc[:,col_number] = X.iloc[:,col_number] / X_sum
X

ypredict,0.0,1.0,2.0
video_name,,,
Test_pour_AFPA,0.338624,0.656085,0.005291
Video_1,0.556291,0.443709,0.000000
WIN_20210323_19_17_40_Pro,0.059524,0.940476,0.000000
WIN_20210329_10_16_02_Pro,0.095238,0.904762,0.000000
WIN_20210330_13_10_29_Pro,0.357616,0.629139,0.013245
WIN_20210331_21_22_52_Pro,0.047059,0.800000,0.152941
WIN_20210402_19_04_53_Pro,0.497143,0.474286,0.028571
WIN_20210403_18_49_15_Pro,0.198895,0.790055,0.011050
WIN_20210404_10_58_27_Pro,0.570000,0.430000,0.000000


In [22]:
#recuperalo stress globale vero 
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.drop(['1', '8', '9', '10', '11', '12', '17', '18'], axis =1)
df_annotations_stress

,video_name,stress_global
0,Test_pour_AFPA,1.0
1,Video_1,0.0
2,WIN_20210323_19_17_40_Pro,1.0
3,WIN_20210329_10_16_02_Pro,1.0
4,WIN_20210330_13_10_29_Pro,0.0
5,WIN_20210331_21_22_52_Pro,1.0
6,WIN_20210402_14_27_50_Pro,1.0
7,WIN_20210402_19_04_53_Pro,2.0
8,WIN_20210403_18_49_15_Pro,2.0
9,WIN_20210404_10_58_27_Pro,1.0


In [23]:
Xy = X.merge(df_annotations_stress, how='inner', on=['video_name'])
X = Xy.iloc[:,:-1].set_index(['video_name'])
y = Xy.iloc[:,-1]
Xy

,video_name,0.0,1.0,2.0,stress_global
0,Test_pour_AFPA,0.338624,0.656085,0.005291,1.0
1,Video_1,0.556291,0.443709,0.000000,0.0
2,WIN_20210323_19_17_40_Pro,0.059524,0.940476,0.000000,1.0
3,WIN_20210329_10_16_02_Pro,0.095238,0.904762,0.000000,1.0
4,WIN_20210330_13_10_29_Pro,0.357616,0.629139,0.013245,0.0
5,WIN_20210331_21_22_52_Pro,0.047059,0.800000,0.152941,1.0
6,WIN_20210402_19_04_53_Pro,0.497143,0.474286,0.028571,2.0
7,WIN_20210403_18_49_15_Pro,0.198895,0.790055,0.011050,2.0
8,WIN_20210404_10_58_27_Pro,0.570000,0.430000,0.000000,1.0
9,WIN_20210404_21_41_12_Pro,0.465517,0.534483,0.000000,0.0


In [24]:
X

,0.0,1.0,2.0
video_name,,,
Test_pour_AFPA,0.338624,0.656085,0.005291
Video_1,0.556291,0.443709,0.000000
WIN_20210323_19_17_40_Pro,0.059524,0.940476,0.000000
WIN_20210329_10_16_02_Pro,0.095238,0.904762,0.000000
WIN_20210330_13_10_29_Pro,0.357616,0.629139,0.013245
WIN_20210331_21_22_52_Pro,0.047059,0.800000,0.152941
WIN_20210402_19_04_53_Pro,0.497143,0.474286,0.028571
WIN_20210403_18_49_15_Pro,0.198895,0.790055,0.011050
WIN_20210404_10_58_27_Pro,0.570000,0.430000,0.000000


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [27]:
best_result_tw5_glob, y_predict_tw5_glob, result_list_tw5_glob = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list,output_predict=True, n_jobs=-1, verbose=True)

Fitting 29 folds for each of 20 candidates, totalling 580 fits
Best estimator LogisticRegression(C=0.01, multi_class='multinomial', random_state=42)
Best results 0.41379310344827586
Best params {'C': 0.01, 'class_weight': None}
accuracy (mean, std) 0.41379310344827586 0.4925123054167483
f1 (mean, std) 0.41379310344827586 0.4925123054167483
balanced accuracy (mean, std) 0.41379310344827586 0.4925123054167483
precision (mean, std) 0.41379310344827586 0.4925123054167483
recall (mean, std) 0.41379310344827586 0.4925123054167483

Fitting 29 folds for each of 80 candidates, totalling 2320 fits
Best estimator Pipeline(steps=[('pca', PCA(n_components=1)),
                ('logistic',
                 LogisticRegression(C=0.01, multi_class='multinomial'))])
Best results 0.41379310344827586
Best params {'logistic__C': 0.01, 'logistic__class_weight': None, 'pca__n_components': 1}
accuracy (mean, std) 0.41379310344827586 0.4925123054167483
f1 (mean, std) 0.41379310344827586 0.4925123054167483
bala

In [28]:
best_result_tw5_glob

{'best_estimator': KNeighborsClassifier(n_neighbors=15),
 'best_score': 0.4482758620689655,
 'best_params': {'n_neighbors': 15, 'p': 2, 'weights': 'uniform'},
 'mean_test_f1_score': 0.4482758620689655,
 'std_test_f1_score': 0.4973174173053778,
 'mean_test_accuracy_score': 0.4482758620689655,
 'std_test_accuracy_score': 0.4973174173053778,
 'mean_test_balanced_accuracy_score': 0.4482758620689655,
 'std_test_balanced_accuracy_score': 0.4973174173053778,
 'mean_test_precision': 0.4482758620689655,
 'std_test_precision': 0.4973174173053778,
 'mean_test_recall': 0.4482758620689655,
 'std_test_recall': 0.4973174173053778}

In [29]:
y_predict_tw5_glob
##ATTENZIONE NON PREDICE MAI I 2

array([0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0.])

## ORA SEMPRE PARTENDO DALLE PEDIZIONI DI  5 SECONDI FACCIO NEL AGGIUNGENDO MINMAX sulle predizioni e predico su global

In [70]:
predict5s['ypredict'].value_counts()

1.0    3342
0.0    1624
2.0     175
Name: ypredict, dtype: int64

In [33]:
predict5s_global = predict5s.drop("diapo", axis = 1)
predict5s_global

,video_name,frameTimeWindow,ypredict
0,Test_pour_AFPA,44,0.0
1,Test_pour_AFPA,45,0.0
2,Test_pour_AFPA,46,0.0
3,Test_pour_AFPA,47,0.0
4,Test_pour_AFPA,48,0.0
...,...,...,...
5136,WIN_20210417_14_53_12_Pro,120,1.0
5137,WIN_20210417_14_53_12_Pro,121,1.0
5138,WIN_20210417_14_53_12_Pro,122,1.0
5139,WIN_20210417_14_53_12_Pro,123,1.0


In [34]:
from scipy.stats import kurtosis, skew

def percentil25(x): 
    return np.percentile(x, q=25)

def percentil75(x): 
    return np.percentile(x, q=75)

In [35]:

X_glob_minmax = predict5s_global.groupby(['video_name']).agg({'ypredict':['mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew]})

In [36]:
X_glob_minmax

ypredict                             \
                                     mean  min  max median       std   
video_name                                                             
Test_pour_AFPA                   0.666667  0.0  2.0    1.0  0.483779   
Video_1                          0.443709  0.0  1.0    0.0  0.498474   
WIN_20210323_19_17_40_Pro        0.940476  0.0  1.0    1.0  0.237310   
WIN_20210329_10_16_02_Pro        0.904762  0.0  1.0    1.0  0.294547   
WIN_20210330_13_10_29_Pro        0.655629  0.0  2.0    1.0  0.503936   
WIN_20210331_21_22_52_Pro        1.105882  0.0  2.0    1.0  0.435782   
WIN_20210402_19_04_53_Pro        0.531429  0.0  2.0    1.0  0.554901   
WIN_20210403_18_49_15_Pro        0.812155  0.0  2.0    1.0  0.419081   
WIN_20210404_10_58_27_Pro        0.430000  0.0  1.0    0.0  0.496318   
WIN_20210404_21_41_12_Pro        0.534483  0.0  1.0    1.0  0.499888   
WIN_20210405_15_09_16_Pro        0.780645  0.0  1.0    1.0  0.415151   
WIN_20210406_15_06_15_Pro        0.731707  0.0  1.0    1.0  0.444155   
WIN_20210406_18_35_52_Pro        0.841772  0.0  1.0    1.0  0.366115   
WIN_20210406_18_49_10_Pro        0.625731  0.0  2.0    1.0  0.497327   
WIN_20210406_21_05_52_Pro        1.059459  0.0  2.0    1.0  0.279226   
WIN_20210407_09_04_05_Pro        0.047120  0.0  2.0    0.0  0.257272   
WIN_20210407_14_54_56_Pro_edit2  0.509434  0.0  1.0    1.0  0.501490   
WIN_20210408_11_48_58_Pro        0.817778  0.0  2.0    1.0  0.479625   
WIN_20210408_14_00_44_Pro        0.589888  0.0  1.0    1.0  0.493241   
WIN_20210408_14_02_19_Pro        0.964706  0.0  1.0    1.0  0.185067   
WIN_20210408_14_11_32_Pro        0.384615  0.0  1.0    0.0  0.488071   
WIN_20210408_15_20_51_Pro        1.313725  0.0  2.0    1.0  0.722344   
WIN_20210408_16_04_32_Pro        0.817647  0.0  2.0    1.0  0.457334   
WIN_20210409_10_26_11_Pro        0.768786  0.0  1.0    1.0  0.422833   
WIN_20210413_15_38_01_Pro        0.722543  0.0  1.0    1.0  0.449043   
WIN_20210414_06_24_52_Pro        0.844560  0.0  2.0    1.0  0.416688   
WIN_20210415_15_41_24_Pro        0.649770  0.0  1.0    1.0  0.478145   
WIN_20210416_08_06_54_Pro        0.932515  0.0  2.0    1.0  0.432176   
WIN_20210417_14_53_12_Pro        0.305344  0.0  1.0    0.0  0.462321   

                                                                              
                                percentil25 percentil75   kurtosis      skew  
video_name                                                                    
Test_pour_AFPA                          0.0         1.0  -1.307851 -0.565240  
Video_1                                 0.0         1.0  -1.948650  0.226606  
WIN_20210323_19_17_40_Pro               1.0         1.0  11.863291 -3.723344  
WIN_20210329_10_16_02_Pro               1.0         1.0   5.605263 -2.757764  
WIN_20210330_13_10_29_Pro               0.0         1.0  -1.142844 -0.338646  
WIN_20210331_21_22_52_Pro               1.0         1.0   1.720146  0.545262  
WIN_20210402_19_04_53_Pro               0.0         1.0  -0.915190  0.381786  
WIN_20210403_18_49_15_Pro               1.0         1.0   0.589966 -1.134212  
WIN_20210404_10_58_27_Pro               0.0         1.0  -1.920033  0.282785  
WIN_20210404_21_41_12_Pro               0.0         1.0  -1.980884 -0.138260  
WIN_20210405_15_09_16_Pro               1.0         1.0  -0.160185 -1.356398  
WIN_20210406_15_06_15_Pro               0.0         1.0  -0.906061 -1.045916  
WIN_20210406_18_35_52_Pro               1.0         1.0   1.507970 -1.872958  
WIN_20210406_18_49_10_Pro               0.0         1.0  -1.491908 -0.375286  
WIN_20210406_21_05_52_Pro               1.0         1.0   8.411619  2.103198  
WIN_20210407_09_04_05_Pro               0.0         0.0  37.109970  5.950587  
WIN_20210407_14_54_56_Pro_edit2         0.0         1.0  -1.998575 -0.037743  
WIN_20210408_11_48_58_Pro               1.0         1.0   0.400208 -0.465113  
WIN_20210408_14_00_44_Pro               0.0         1.0  -1.866406 -0

In [38]:
#questo e' lo stress vero per diapositiva
#recuperalo stress globale vero 
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.drop(['1', '8', '9', '10', '11', '12', '17', '18'], axis =1)
df_annotations_stress

,video_name,stress_global
0,Test_pour_AFPA,1.0
1,Video_1,0.0
2,WIN_20210323_19_17_40_Pro,1.0
3,WIN_20210329_10_16_02_Pro,1.0
4,WIN_20210330_13_10_29_Pro,0.0
5,WIN_20210331_21_22_52_Pro,1.0
6,WIN_20210402_14_27_50_Pro,1.0
7,WIN_20210402_19_04_53_Pro,2.0
8,WIN_20210403_18_49_15_Pro,2.0
9,WIN_20210404_10_58_27_Pro,1.0


In [39]:
X_glob_minmaxy = X_glob_minmax.merge(df_annotations_stress, how='inner', on=['video_name'])
X_glob_minmax = X_glob_minmaxy.iloc[:,:-1].set_index(['video_name'])
y_glob_minmax = X_glob_minmaxy.iloc[:,-1]
X_glob_minmaxy

,video_name,"(ypredict, mean)","(ypredict, min)","(ypredict, max)","(ypredict, median)","(ypredict, std)","(ypredict, percentil25)","(ypredict, percentil75)","(ypredict, kurtosis)","(ypredict, skew)",stress_global
0,Test_pour_AFPA,0.666667,0.0,2.0,1.0,0.483779,0.0,1.0,-1.307851,-0.565240,1.0
1,Video_1,0.443709,0.0,1.0,0.0,0.498474,0.0,1.0,-1.948650,0.226606,0.0
2,WIN_20210323_19_17_40_Pro,0.940476,0.0,1.0,1.0,0.237310,1.0,1.0,11.863291,-3.723344,1.0
3,WIN_20210329_10_16_02_Pro,0.904762,0.0,1.0,1.0,0.294547,1.0,1.0,5.605263,-2.757764,1.0
4,WIN_20210330_13_10_29_Pro,0.655629,0.0,2.0,1.0,0.503936,0.0,1.0,-1.142844,-0.338646,0.0
5,WIN_20210331_21_22_52_Pro,1.105882,0.0,2.0,1.0,0.435782,1.0,1.0,1.720146,0.545262,1.0
6,WIN_20210402_19_04_53_Pro,0.531429,0.0,2.0,1.0,0.554901,0.0,1.0,-0.915190,0.381786,2.0
7,WIN_20210403_18_49_15_Pro,0.812155,0.0,2.0,1.0,0.419081,1.0,1.0,0.589966,-1.134212,2.0
8,WIN_20210404_10_58_27_Pro,0.430000,0.0,1.0,0.0,0.496318,0.0,1.0,-1.920033,0.282785,1.0
9,WIN_20210404_21_41_12_Pro,0.534483,0.0,1.0,1.0,0.499888,0.0,1.0,-1.980884,-0.138260,0.0


In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X_glob_minmax.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X_glob_minmax, y_glob_minmax, groups)

In [41]:
best_result_tw5_glob_minmax, y_predict_tw5_glob_minmax, result_list_tw5_glob_minmax = runGridSearchClassifiers(X_glob_minmax, y_glob_minmax, cv_loo, models_list, parameters_list,output_predict=True, n_jobs=-1, verbose=True)

Fitting 29 folds for each of 20 candidates, totalling 580 fits
Best estimator LogisticRegression(C=0.05, multi_class='multinomial', random_state=42)
Best results 0.4482758620689655
Best params {'C': 0.05, 'class_weight': None}
accuracy (mean, std) 0.4482758620689655 0.4973174173053778
f1 (mean, std) 0.4482758620689655 0.4973174173053778
balanced accuracy (mean, std) 0.4482758620689655 0.4973174173053778
precision (mean, std) 0.4482758620689655 0.4973174173053778
recall (mean, std) 0.4482758620689655 0.4973174173053778

Fitting 29 folds for each of 80 candidates, totalling 2320 fits
Best estimator Pipeline(steps=[('pca', PCA(n_components=1)),
                ('logistic',
                 LogisticRegression(C=0.01, class_weight='balanced',
                                    multi_class='multinomial'))])
Best results 0.4827586206896552
Best params {'logistic__C': 0.01, 'logistic__class_weight': 'balanced', 'pca__n_components': 1}
accuracy (mean, std) 0.4827586206896552 0.4997026464203254

In [42]:
best_result_tw5_glob_minmax

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=1)),
                 ('logistic',
                  LogisticRegression(C=0.01, class_weight='balanced',
                                     multi_class='multinomial'))]),
 'best_score': 0.4827586206896552,
 'best_params': {'logistic__C': 0.01,
  'logistic__class_weight': 'balanced',
  'pca__n_components': 1},
 'mean_test_f1_score': 0.4827586206896552,
 'std_test_f1_score': 0.4997026464203254,
 'mean_test_accuracy_score': 0.4827586206896552,
 'std_test_accuracy_score': 0.4997026464203254,
 'mean_test_balanced_accuracy_score': 0.4827586206896552,
 'std_test_balanced_accuracy_score': 0.4997026464203254,
 'mean_test_precision': 0.4827586206896552,
 'std_test_precision': 0.4997026464203254,
 'mean_test_recall': 0.4827586206896552,
 'std_test_recall': 0.4997026464203254}

In [43]:
y_predict_tw5_glob_minmax

array([0., 0., 2., 2., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2., 1., 0.,
       0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 1., 0.])